In [1]:
import my_funcs as f
import pandas as pd

In [2]:
import harmonica as hm

In [3]:
import tqdm 
from tqdm.notebook import trange

In [4]:
%matplotlib widget

In [5]:
import math
import numpy as np

import geopandas as gpd
import pandas as pd

import pyproj
from shapely import geometry

import verde as vd
import rioxarray as rio

import matplotlib.pyplot as plt
import seaborn as sns

####################################### LEMBRAR DE PEDIR AJUDA
import tqdm 
from tqdm.notebook import trange


# DEFININDO CAMINHO PARA A BASE DE DADOS
gdb = '/home/ggrl/geodatabase/'


# ---------------------------------------- DEFININDO FUNÇÕES PARA SCRIPT ---------------------------------------#

# IMPORTADOR DE DADOS AEROGEOFISICOS
def importar_geof(raw_data):
    geof_dataframe = pd.read_csv(gdb+'/geof/'+str(raw_data))
    return geof_dataframe


# IMPORTADOR DE LITOLOGIAS POR ESCALA --------------------------------------------------------------------------#
def importar(camada, mapa=False):
    lito =  gpd.read_file(gdb+'database.gpkg',
                        driver= 'GPKG',
                        layer= camada)
    if mapa:
        folha = lito[lito.MAPA == 'Carta geológica da folha '+mapa]
        return(folha)
    else:
        return(lito)

    
# DEFININDO LIMITES DE CADA FOLHA CARTOGRÁFICA -----------------------------------------------------------------#
def regions(gdf):
    # CRIANDO COLUNA REGION EM COORDENADAS GEOGRÁFICAS
    bounds = gdf.bounds
    #print(bounds[:1])

    gdf['region'] = \
    [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                              bounds['miny'],bounds['maxy'])]

    # CRIANDOCOLUNA REGIONS EM COORDENADAS PROJETADAS
    gdf = gdf.to_crs("EPSG:32723")
    #print(f"{gdf.crs}")
    
    bounds = gdf.bounds
    #print(bounds[:1])
    
    gdf['region_proj'] = \
    [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                          bounds['miny'],bounds['maxy'])]
    return gdf


# Nomeador de Grids ------------------------------------------------------------------------------------------#
def nomeador_grid(left,right,top,bottom,escala=5):
    e1kk=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    e500k=[['V','Y'],['X','Z']]
    e250k=[['A','C'],['B','D']]
    e100k=[['I','IV'],['II','V'],['III','VI']]
    e50k=[['1','3'],['2','4']]
    e25k=[['NW','SW'],['NE','SE']]

    if left>right:
        print('Oeste deve ser menor que leste')
    if top<bottom:
        print('Norte deve ser maior que Sul')
    
    else:
        id_folha=''
        if top<=0:
            id_folha+='S'
            north=False
            index=math.floor(-top/4)
        else:
            id_folha+='N'
            north=True
            index=math.floor(bottom/4)
        
        numero=math.ceil((180+right)/6)
        id_folha+=e1kk[index]+str(numero)

        lat_gap=abs(top-bottom)
        #p500k-----------------------
        if (lat_gap<=2) & (escala>=1):
            LO=math.ceil(right/3)%2==0
            NS=math.ceil(top/2)%2!=north
            id_folha+='_'+e500k[LO][NS]
        #p250k-----------------------
        if (lat_gap<=1) & (escala>=2):
            LO=math.ceil(right/1.5)%2==0
            NS=math.ceil(top)%2!=north
            id_folha+='_'+e250k[LO][NS]
        #p100k-----------------------
        if (lat_gap<=0.5) & (escala>=3):
            LO=(math.ceil(right/0.5)%3)-1
            NS=math.ceil(top/0.5)%2!=north
            id_folha+='_'+e100k[LO][NS]
        #p50k------------------------
        if (lat_gap<=0.25) & (escala>=4):
            LO=math.ceil(right/0.25)%2==0
            NS=math.ceil(top/0.25)%2!=north
            id_folha+='_'+e50k[LO][NS]
        #p25k------------------------
        if (lat_gap<=0.125) & (escala>=5):
            LO=math.ceil(right/0.125)%2==0
            NS=math.ceil(top/0.125)%2!=north
            id_folha+='_'+e25k[LO][NS]
        return id_folha


# DEFININDO NOMES DA MALHA A PARTIR DA ARTICULA~AO SISTEMÁTICA DE FOLHAS DE CARTAS. 
# CONSTURINDO UMA LISTA E DEFININDO COMO UMA SERIES (OBJETO DO PANDAS).
def nomeador_malha(gdf):
    df = pd.DataFrame(gdf)
    lista_malha = []
    for index, row in df.iterrows():
        row['id_folha'] = (nomeador_grid(row.region[0],row.region[1],
                                         row.region[3],row.region[2],escala=5))
        lista_malha.append(row.id_folha)

    gdf['id_folha'] = lista_malha


# SELECIONADOR DE REGIÃO  ------------------------------------------------------------------------------------------#
def select_area(escala,id):
    malha_cartog = importar('malha_cartog_'+escala+'_wgs84')
    malha_cartog_gdf_select = malha_cartog[malha_cartog['id_folha'].str.contains(id)]       # '.contains' não é ideal.
    malha_cartog_gdf_select = regions(malha_cartog_gdf_select)    
    
    return(malha_cartog_gdf_select)

# LISTANDO REGIÕES DE CADA FOLHA DE CARTAS DA MALHA CARTOGRÁFICA \ ['REGEION'] = ['ID_FOLHA'] REDUNDANCIA
def cartas(escala,id):
    # SELECIONANDO AREA DE ESTUDO
    if escala and id:
        print('# --- Iniciando seleção de área de estudo')
        malha_cartog_gdf_select = select_area(escala,id)
        #print("Indexando a coluna 'id_folha'")
        malha_cartog_gdf_select.set_index('id_folha',inplace=True)
        print(malha_cartog_gdf_select.index)

   
    # CRIANDO UM DICIONÁRIO DE CARTAS
    #print(f"Retirada da coluna 'geometry'")
    malha_cartog_df_select = malha_cartog_gdf_select.drop(columns=['geometry'])
    malha_cartog_df_select['raw_data'] =''
    malha_cartog_df_select['interpolado'] =''
    malha_cartog_df_select['scores'] =''
    malha_cartog_df_select['lito_geof'] =''
    malha_cartog_df_select['mean_score'] =''
    print("Gerando dicionário com o index")
    dic_cartas = malha_cartog_df_select.to_dict()
    print(dic_cartas.keys())
    #print(dic_cartas)
    
    # APENAS UMA FOLHA DE CARTA SELECIONADA
    if len(dic_cartas['raw_data']) > 1:
        print(f"{len(dic_cartas['raw_data'])} folhas cartográfica selecionadas")
        print("")

    # MAIS DE UMA FOLHA DE CARTA SELECIONADA
    if len(dic_cartas['raw_data']) == 1:
        print(f"{len(dic_cartas['raw_data'])} folha cartográfica selecionada")
        print("")
    return malha_cartog_gdf_select, dic_cartas


# LISTANDO ATRIBUTOS GEOFÍSICOS E ATRIBUTOS GEOGRÁFICOS
def list_atributos(geof):
    atributos_geof = list(geof.columns)
    lista_atributo_geof=[]
    lista_atributo_geog=[]
    lista_atributo_proj=[]

    for atributo in atributos_geof:
        if atributo == 'LATITUDE':
            lista_atributo_geog.append(atributo)
        elif atributo == 'LONGITUDE':
            lista_atributo_geog.append(atributo)
        elif atributo == 'LONG':
            lista_atributo_geog.append(atributo)
        elif atributo == 'LAT':
            lista_atributo_geog.append(atributo) 
        elif atributo == 'X':
            lista_atributo_proj.append(atributo)
        elif atributo == 'Y':
            lista_atributo_proj.append(atributo)
        elif atributo == 'UTME':
            lista_atributo_proj.append(atributo)
        elif atributo == 'UTMN':
            lista_atributo_proj.append(atributo)
        else:
            lista_atributo_geof.append(atributo)
    codigo=str(geof)        
    print(f"# --- # Listagem de dados do aerolevantamento:  ")
    print(f"Lista de atributos geofísicos = {lista_atributo_geof}")
    print(f"lista de atributos geograficos = {lista_atributo_geog}")
    print(f"lista de atributos projetados = {lista_atributo_proj}")
    return lista_atributo_geof, lista_atributo_geog, lista_atributo_proj


# DESCRIÇÃO DOS DADOS AEROGEOFÍSICOS
def descricao(geof):            
    lista_atributo_geof,lista_atributo_geog,lista_atributo_proj = list_atributos(geof)  # USANDO FUNCAO DEFINIDA ACIMA PARA CATEGORIZAR METADADO
    
    datadict = pd.DataFrame(geof.dtypes)
    datadict["Valores Faltantes"] = geof.isnull().sum()
    datadict["Valores Únicos"] = geof.nunique()
    datadict["Amostragem"] = geof.count()
    datadict = datadict.rename(columns = {0 : 'dType'})

    lista_negativo=[]
    for i in lista_atributo_geof:
        lista_negativo.append(geof.query(i+' < 0')[i].count())


    geof_df = geof.drop(axis=0,columns=lista_atributo_geog)
    geof_df.drop(axis=0,columns=lista_atributo_proj,inplace=True)

    #datadict['Valores Negativos'] = lista_negativo

    geof_df_describe = geof_df.describe(percentiles=[0.001,0.1,0.25,0.5,0.75,0.995])
    
    return datadict,lista_atributo_geof,lista_atributo_geog,lista_atributo_proj,geof_df_describe


######################################################################################################################################
# # --------------------- DEFININDO FUNÇÃO DE QUE CHAMARÁ AS FUNÇÕES ANTERIORES PROVOCANDO UM ENCADEAMENTO DE OPERAÇÕES -------------- 
def interpolar(escala,id,geof,
               standard_verde=None,psize=None,spacing=None,degree=None,n_splits=None,
               camada=None,nome=None,crs__='proj',describe=False):
    # DEFININDO PADRÃO DE INTERPOLAÇÃO VERDE
    if standard_verde:
        save=None
        degree=1
        spacing=499
        psize= 100
    else:
        save=None
        degree=degree
        spacing=spacing
        psize=psize
                
    # LISTANDO REGIOES DAS FOLHAS DE CARTAS
    malha_cartog_gdf_select, dic_cartas = cartas(escala,id)
        
    # DESCREVENDO ATRIBUTOS ESTATISTICOS DOS DADOS    
    datadict,lista_atributo_geof,lista_atributo_geog,lista_atributo_proj,geof_df_describe = descricao(geof)

    # ITERANDO ENTRE AS FOLHAS DE CARTAS
    print("")
    print(f"# --- Início da iteração entre as folhas cartográficas #")

    for index, row in malha_cartog_gdf_select.iterrows():
        # RECORTANDO DATA PARA CADA FOLHA COM ['region.proj']
        data = geof[vd.inside((geof.X, geof.Y), region = row.region_proj)]

        # REMOVENDO VALORES NEGATIVOS DOS DADOS AEROGAMAESPECTOMETRICOS
        data.loc[data.KPERC < 0, 'KPERC'] = 0
        data.loc[data.eU < 0, 'eU'] = 0
        data.loc[data.eTH < 0, 'eTH'] = 0

        a,b,c,d,e = descricao(data)

        # CALCULANDO RAZOES DE BANDAS PARA OS DADOS
        lista_atributo_geof.append('U_TH')
        lista_atributo_geof.append('U_K')
        lista_atributo_geof.append('TH_K')


        data_razoes = data

        data_razoes.loc[data_razoes.eU < (e['eU']['mean']) / 10, 'eU'] = (e['eU']['mean']) / 10
        data_razoes.loc[data_razoes.KPERC < (e['KPERC']['mean']) / 10, 'KPERC'] = (e['KPERC']['mean']) / 10
        data_razoes.loc[data_razoes.eTH   < (e['eTH']['mean'])   / 10, 'eTH'] = (e['eTH']['mean']) / 10

        data['U_TH'] = data_razoes.eU / data_razoes.eTH
        data['U_K']  = data_razoes.eU  / data_razoes.KPERC
        data['TH_K']  = data_razoes.eTH / data_razoes.KPERC


        # GERANDO TUPLA DE COORDENADAS
        if data.empty:
            None
            
        elif len(data) < 1000:
            None
            #print(f"A folha {index} possui apenas '{len(data)}' pontos coletados que devem ser adicionados a folha mais próxima")
            
        else:
            print(f"# Folha de código: {index}")
            print(f" Atualizando dados brutos em dic_cartas['raw_data']")
            x = {index:data}
            dic_cartas['raw_data'].update(x) 
            print(f" com {len(data)} pontos de contagens radiométricas coletados com linhas de voo de {spacing} metros")
            
            # ADICIONANDO ATRIBUTOS GEOFÍSICOS AO DICIONÁRIO INTERPOLADO
            interpolado={}
            for atributo in lista_atributo_geof:
                x = {atributo:''}
                interpolado.update(x)
            #print(f" Construindo dic_cartas['interpolado'] vazio com os atributos geofísicos")
            #print(interpolado.keys())
                
            # ADICIONANDO ATRIBUTOS AO DICIONÁRIO SCORES
            scores={}
            for atributo in lista_atributo_geof:
                y = {atributo:''}
                scores.update(y)

            mean_score={}
            for atributo in lista_atributo_geof:
                x = {atributo:''}
                mean_score.update(x)    

            #print(f" Construindo dicionário vazio de score do cross validation")
            #print(scores.keys())

            # Iterando entre os canais de interpolação
            print("")
            print(f"# --- Inicio da interpolação com verde Splines # ")
            for i in lista_atributo_geof:
                # Definindo encadeamento de processsos para interpolação
                chain = vd.Chain([
                                ('trend', vd.Trend(degree=degree)),
                                ('reduce', vd.BlockReduce(np.median, spacing=spacing)),
                                ('spline', vd.Spline(dampings=dampings,
                                                     mindist=mindist))
                            ])
                
                print(f"Encadeamento: {chain}") 
                coordinates = (data.X.values, data.Y.values)
                
                # ESCOLHER MELHOR TRAIN TEST SPLIT


                print(f"Fitting Model of  '{i}' variable...")
                chain.fit(coordinates, data[i])

                # Griding the predicted data.
                print(f"Predicting values of '{i}' to a regular grid of {psize} m")
                grid = chain.grid(spacing=psize, data_names=[i],pixel_register=True)
                interpolado[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)
                
                # ATUALIZAÇÃO DE DICIONÁRIO DE INTERPOLADOS
                x = {index:interpolado}
                dic_cartas['interpolado'].update(x)
                print(f" Dicionário de dados interpolados da folha {index} atualizados")

                # Processo de validação cruzada da biblioteca verde
                if n_splits:
                    cv     = vd.BlockKFold(spacing=spacing,
                                n_splits=n_splits,
                                shuffle=True)
                    print(f"Parâmetros de validação cruzada: {cv}")

                    scores[i] = vd.cross_val_score(chain,
                                            coordinates,
                                            data[i],
                                            cv=cv,
                                            delayed=True)

                    import dask
                    mean_score[i] = dask.delayed(np.mean)(scores[i])
                    #print("Delayed mean:", mean_score)

                    mean_score[i] = mean_score[i].compute()
                    print("Mean score:", mean_score)


                # ATUALIZAÇÃO DE DICIONÁRIO DE SCORES
                #print(f" Atualizando dicionário com scores")
                y = {index:scores}
                dic_cartas['scores'].update(y)

                x = {index:mean_score}
                dic_cartas['mean_score'].update(x)
                print(f"# Folha {index} atualizada ao dicionário")

                
                # SALVANDO DADOS INTERPOLADOS NO FORMATO .TIF
                if save:
                    local='grids/geof_'+str(save)+'_'+str(psize)+'m_'+i+'_'+row.id_folha+'.tif'
                    print('salvando grids/geof_'+str(save)+'_'+str(psize)+'m_'+i+'_'+row.id_folha+'.tif')
                    #grids[i].to_netcdf(gdb+'/grids/geof_3022_'+str(psize)+'m_'+i+'_'+row.id_folha+'.nc')
                    tif_ = interpolado[i].rename(easting = 'x',northing='y')
                    tif_.rio.to_raster(gdb+local)
        
            # RETIRANDO VALORES DE LITOLOGIA DE CADA PIXEL
            if describe:
                print("")
                print(f"# --- Inicio da análise geoestatística")
                lista_interpolado = list()

                for i in lista_atributo_geof:
                    df = interpolado[i].to_dataframe()
                    lista_interpolado.append(df[i])

                geof_grids = pd.concat(lista_interpolado,axis=1, join='inner')
                geof_grids.reset_index(inplace=True)
                geof_grids['geometry'] =\
                     [geometry.Point(x,y) for x, y in zip(geof_grids['easting'], geof_grids['northing'])]

                print('Ajustando crs')
                if crs__=='proj':
                    gdf = gpd.GeoDataFrame(geof_grids,crs=32723)
                    gdf = gdf.set_crs(32723, allow_override=True)
                    gdf = gdf.to_crs("EPSG:32723")
                    print(f" geof: {gdf.crs}")
                else:
                    gdf = gpd.GeoDataFrame(geof_grids,crs=32723)
                    gdf = gdf.set_crs(32723, allow_override=True)
                    gdf = gdf.to_crs("EPSG:4326")
                    print(f" geof: {gdf.crs}")

                # IMPORTANDO VETORES LITOLÓGICOS
                litologia = importar(camada,nome)
                litologia.reset_index(inplace=True)
                if crs__=='proj':
                    litologia = litologia.set_crs(32723, allow_override=True)
                    litologia = litologia.to_crs("EPSG:32723")
                    print(f" lito: {litologia.crs}")
                else:
                    litologia = litologia.set_crs(32723, allow_override=True)
                    litologia = litologia.to_crs("EPSG:4326")
                    litologia=litologia.cx[row.region[0]:row.region[1],row.region[2]:row.region[3]]
                    litologia.reset_index(inplace=True)
                    print(f" lito: {litologia.crs}")


                print(f"# -- Calculando geometria mais próxima para cada um dos {len(geof_grids)} centróides de pixel")
                #print(f"# Listagem de unidade geológicas do mapa litologia['MAPA'].unique():")
                #print(f" {list(litologia['litologia'].unique())}")
                lito_geof = geof_grids
                lito_geof['closest_unid'] = gdf['geometry'].apply(lambda x: litologia['litologia'].iloc[litologia.distance(x).idxmin()])
                print(f"# Listagem de unidades geológicas presentes na folha de id {index}:    ")
                print(f"  {list(lito_geof['closest_unid'].unique())}")

                # Adicionando lito_geof ao dicionario
                print('')
                print(f" Adicionando dataframe com valores de litologia e geofíscios ao dicionário de cartas")
                x = {index:lito_geof}
                dic_cartas['lito_geof'].update(x)
                #print(dic_cartas['lito_geof'][index].keys())

                print('__________________________________________')
            print(" ")
    print("Dicionário de cartas disponível")
    return dic_cartas




In [8]:
def get_region(escala,id,geof):
    geof_dataframe = importar_geof(geof)
    
    # LISTANDO REGIOES DAS FOLHAS DE CARTAS
    malha_cartog_gdf_select, dic_cartas = cartas(escala,id)
        
    # DESCREVENDO ATRIBUTOS ESTATISTICOS DOS DADOS    
    datadict,lista_atributo_geof,lista_atributo_geog,lista_atributo_proj,geof_df_describe = descricao(geof_dataframe)

    # ITERANDO ENTRE AS FOLHAS DE CARTAS
    print("")
    print(f"# --- Início da iteração entre as folhas cartográficas #")

    for index, row in malha_cartog_gdf_select.iterrows():
        # RECORTANDO DATA PARA CADA FOLHA COM ['region.proj']
        data = geof_dataframe[vd.inside((geof_dataframe.X, geof_dataframe.Y), region = row.region_proj)]
        #del geof_dataframe
        # GERANDO TUPLA DE COORDENADAS
        if data.empty:
            None
            
        elif len(data) < 1000:
            None
            #print(f"A folha {index} possui apenas '{len(data)}' pontos coletados que devem ser adicionados a folha mais próxima")
            
        else:
            print(f"# Folha de código: {index}")
            print(f" Atualizando dados brutos em dic_cartas['raw_data']")
            x = {index:data}
            dic_cartas['raw_data'].update(x) 
            print(f" com {len(data)} pontos")
            
        
    return dic_cartas

In [9]:
dic_cartas = get_region('25k','SF23_Y_A_III_4_SE','mag_1105')

# --- Iniciando seleção de área de estudo


/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


Index(['SF23_Y_A_III_4_SE'], dtype='object', name='id_folha')
Gerando dicionário com o index
dict_keys(['region', 'region_proj', 'raw_data', 'interpolado', 'scores', 'lito_geof', 'mean_score'])
1 folha cartográfica selecionada

# --- # Listagem de dados do aerolevantamento:  
Lista de atributos geofísicos = ['ALTURA', 'MDT', 'ALTURA_1', 'MAGIGRF']
lista de atributos geograficos = ['LATITUDE', 'LONGITUDE']
lista de atributos projetados = ['X', 'Y']

# --- Início da iteração entre as folhas cartográficas #
# Folha de código: SF23_Y_A_III_4_SE
 Atualizando dados brutos em dic_cartas['raw_data']
 com 49331 pontos


In [15]:
data = dic_cartas['raw_data']['SF23_Y_A_III_4_SE']

In [26]:
data

,ALTURA,X,Y,MDT,MAGIGRF
672041,376.49,332876.04,7524887.92,781.03,49.578
672042,378.10,332875.81,7524880.08,781.35,49.661
672043,378.76,332875.58,7524872.25,781.65,49.738
672044,378.37,332875.35,7524864.42,781.92,49.811
672045,377.60,332875.12,7524856.57,782.05,49.886
...,...,...,...,...,...
12085077,115.43,345674.28,7515612.00,824.09,95.717
12085078,117.38,345681.35,7515611.83,823.98,95.870
12085079,119.37,345688.42,7515611.67,823.97,96.026
12085080,121.40,345695.48,7515611.50,824.05,96.183


In [27]:
data.drop(columns=['MDT'],inplace=True)

In [28]:
plt.figure()
tmp = plt.scatter(data.LONGITUDE, data.LATITUDE, c=data.MAGIGRF, s=1)
plt.gca().set_aspect("equal")
plt.colorbar(tmp, label="mGal")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'DataFrame' object has no attribute 'LONGITUDE'

In [10]:
plt.figure()
tmp = plt.scatter(data.X, data.Y, c=data.MAGIGRF, s=1)
plt.gca().set_aspect("equal")
plt.colorbar(tmp, label="nT")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()
tmp = plt.scatter(data.X, data.Y, c=data.MDT, s=1)
plt.gca().set_aspect("equal")
plt.colorbar(tmp, label="m")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
print("Number of data points:", data.shape[0])
print("Mean height of observations:", data.ALTURA.mean())
print("Mean height of observations:", data.ALTURA_1.mean())

Number of data points: 49331
Mean height of observations: 183.26372787902017
Mean height of observations: 176.38948146195952


In [13]:
coordinates = (data.X, data.Y, data.ALTURA)

In [14]:
eql = hm.EQLHarmonic(relative_depth=1000, damping=1)

In [15]:
eql.fit(coordinates, data.MAGIGRF)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [1]:
print("R² score:", eql.score(coordinates, data.MAGIGRF))

NameError: name 'eql' is not defined

In [ ]:
grid = eql.grid(upward=1500, spacing=100, data_names=["magnetic_anomaly"])

In [ ]:
print("\nGenerated grid:\n", grid)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 9), sharey=True)

In [ ]:
maxabs = vd.maxabs(data.MAGIGRF, grid.magnetic_anomaly.values)

ax1.set_title("Observed magnetic anomaly data")
tmp = ax1.scatter(
    data.X,
    data.Y,
    c=data.MAGIGRF,
    s=20,
    vmin=-maxabs,
    vmax=maxabs,
    cmap="seismic",
)
plt.colorbar(tmp, ax=ax1, label="nT", pad=0.05, aspect=40, orientation="horizontal")
ax1.set_xlim(data.X.min(), data.X.max())
ax1.set_ylim(data.Y.min(), data.Y.max())

ax2.set_title("Gridded and upward-continued")
tmp = grid.magnetic_anomaly.plot.pcolormesh(
    ax=ax2,
    add_colorbar=False,
    add_labels=False,
    vmin=-maxabs,
    vmax=maxabs,
    cmap="seismic",
)
plt.colorbar(tmp, ax=ax2, label="nT", pad=0.05, aspect=40, orientation="horizontal")
ax2.set_xlim(data.X.min(), data.X.max())
ax2.set_ylim(data.Y.min(), data.Y.max())

plt.show()